In [5]:
import requests
import json
import time

In [6]:
# Step 1: Fetch Device IDs
url_device_ids = 'https://client-device-service.manhole-metrics.com/client_device_ids'
headers = {'api_key': 'b0dda244-4adc-4215-b9ff-54403dc736c2'}

In [7]:
# Send GET request to get device IDs
response_ids = requests.get(url_device_ids, headers=headers)
print(response_ids)

<Response [200]>


In [8]:
if response_ids.status_code == 200:
    # Step 2: Loop through each device_id and fetch the data for each device
    device_ids = response_ids.json()['device_ids']  # Assuming response returns a list of device IDs
    devices_data = {}

    for device_id in device_ids:
        timestamp = 0
        # The API only returns up to 1000 water levels after the given timestamp, so need to run a loop to get all the data
        while True:
            url_device_data = f'https://client-device-service.manhole-metrics.com/client_device?device_id={device_id}&starting_unix_timestamp={timestamp}'
            print(f'GET {url_device_data}')
            response_device = requests.get(url_device_data, headers=headers)

            if response_device.status_code == 200:
                # Process and display the data for each device
                device_data = response_device.json()
                if (devices_data.get(device_data['device_id'])):
                    devices_data[device_data['device_id']]['water_level_measurements'] += device_data['water_level_measurements']
                else:
                    devices_data[device_data['device_id']] = device_data
                print(f"Device ID: {device_data['device_id']}")
                print(f"Device Coordinates: {device_data['device_coordinates']}")
                print(f"Max Distance: {device_data['max_distance']} mm")
                print(f"Last Water Level: {device_data['last_water_level']} mm")
                print(f"Last Fill Percentage: {device_data['last_fill_percentage']}%")
                print("Water Level Measurements:")
                for measurement in device_data['water_level_measurements']:
                    print(f"  Timestamp: {measurement['measurement_unix_timestamp']}, Water Level: {measurement['water_level_mm']} mm")
                print("\n" + "-"*50 + "\n")
                time.sleep(1)

                if len(device_data['water_level_measurements']):
                    timestamp = 1 + device_data['water_level_measurements'][-1]['measurement_unix_timestamp']
                else:
                    break
            else:
                # If you encounter an error code 500, try running the script again
                print(f"Failed to fetch data for device ID {device_id}. Status code: {response_device.status_code}")

    # Save the fetched data to a JSON file
    with open('devices_data.json', 'w') as f:
        json.dump(devices_data, f)
else:
    # If you encounter an error code 500, try running the script again
    print(f"Failed to fetch device IDs. Status code: {response_ids.status_code}")

GET https://client-device-service.manhole-metrics.com/client_device?device_id=951&starting_unix_timestamp=0
Device ID: 951
Device Coordinates: [37.89500045776367, -122.28700256347656]
Max Distance: 2108 mm
Last Water Level: 208 mm
Last Fill Percentage: 9%
Water Level Measurements:
  Timestamp: 1748377182, Water Level: 128.0 mm
  Timestamp: 1748377262, Water Level: 118.0 mm
  Timestamp: 1748378261, Water Level: 1248.0 mm
  Timestamp: 1748378650, Water Level: 138.0 mm
  Timestamp: 1748378956, Water Level: 158.0 mm
  Timestamp: 1748379262, Water Level: 108.0 mm
  Timestamp: 1748379568, Water Level: 158.0 mm
  Timestamp: 1748379873, Water Level: 128.0 mm
  Timestamp: 1748380179, Water Level: 128.0 mm
  Timestamp: 1748380487, Water Level: 128.0 mm
  Timestamp: 1748380801, Water Level: 48.0 mm
  Timestamp: 1748381786, Water Level: 98.0 mm
  Timestamp: 1748382692, Water Level: 0.0 mm
  Timestamp: 1748383598, Water Level: 118.0 mm
  Timestamp: 1748384511, Water Level: 138.0 mm
  Timestamp: 174